# Delta Table creation for Sensor Data

In [0]:
%sql
drop table genai_usecases.multi_step_tools.sensor_readings_delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS genai_usecases.multi_step_tools.sensor_readings_delta (
  sensor_id STRING,
  ts TIMESTAMP,
  value DOUBLE
)
USING DELTA;

-- Insert sensor readings across multiple days
INSERT INTO genai_usecases.multi_step_tools.sensor_readings_delta (sensor_id, ts, value) VALUES
-- Day 1 readings
('sensor_A', TIMESTAMP '2025-10-21 08:00:00', 22.3),
('sensor_A', TIMESTAMP '2025-10-21 09:00:00', 23.7),
('sensor_A', TIMESTAMP '2025-10-21 10:00:00', 24.0),
('sensor_A', TIMESTAMP '2025-10-21 11:00:00', 999.9),  -- anomaly spike
('sensor_B', TIMESTAMP '2025-10-21 08:30:00', 49.5),
('sensor_B', TIMESTAMP '2025-10-21 09:30:00', 50.2),

-- Day 2 readings
('sensor_A', TIMESTAMP '2025-10-22 08:00:00', 22.1),
('sensor_A', TIMESTAMP '2025-10-22 09:15:00', 23.5),
('sensor_A', TIMESTAMP '2025-10-22 10:30:00', -5.0),   -- invalid reading
('sensor_B', TIMESTAMP '2025-10-22 08:45:00', 51.0),
('sensor_B', TIMESTAMP '2025-10-22 09:45:00', 49.8),
('sensor_B', TIMESTAMP '2025-10-22 10:45:00', 52.3),

-- Day 3 readings
('sensor_A', TIMESTAMP '2025-10-23 08:10:00', 23.0),
('sensor_A', TIMESTAMP '2025-10-23 09:10:00', 24.5),
('sensor_A', TIMESTAMP '2025-10-23 10:10:00', 25.1),
('sensor_B', TIMESTAMP '2025-10-23 08:40:00', 48.7),
('sensor_B', TIMESTAMP '2025-10-23 09:40:00', 50.1),
('sensor_B', TIMESTAMP '2025-10-23 10:40:00', 5000.0),

-- Today readings
('sensor_A', TIMESTAMP '2025-10-24 18:10:00', 27.0),
('sensor_A', TIMESTAMP '2025-10-24 19:10:00', 24.5),
('sensor_A', TIMESTAMP '2025-10-24 12:10:00', 26.1),
('sensor_B', TIMESTAMP '2025-10-24 14:40:00', 23.7),
('sensor_B', TIMESTAMP '2025-10-24 16:40:00', 98.1),
('sensor_B', TIMESTAMP '2025-10-24 20:40:00', 640.0);  -- crazy outlier


In [0]:
%sql
select * from genai_usecases.multi_step_tools.sensor_readings_delta

sensor_id,ts,value
sensor_A,2025-10-21T08:00:00.000Z,22.3
sensor_A,2025-10-21T09:00:00.000Z,23.7
sensor_A,2025-10-21T10:00:00.000Z,24.0
sensor_A,2025-10-21T11:00:00.000Z,999.9
sensor_B,2025-10-21T08:30:00.000Z,49.5
sensor_B,2025-10-21T09:30:00.000Z,50.2
sensor_A,2025-10-22T08:00:00.000Z,22.1
sensor_A,2025-10-22T09:15:00.000Z,23.5
sensor_A,2025-10-22T10:30:00.000Z,-5.0
sensor_B,2025-10-22T08:45:00.000Z,51.0


In [0]:
spark.sql(f"""
            SELECT MAX(value) AS max_value 
            FROM genai_usecases.multi_step_tools.sensor_readings_delta 
            WHERE sensor_id = 'sensor_B' AND to_date(ts) = to_date('2025-10-23')
        """).display()

In [0]:
!pip install langchain databricks-langchain --quiet

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import os
from databricks_langchain import ChatDatabricks
from langchain.tools import tool
from langchain_core.messages import HumanMessage,SystemMessage
from datetime import datetime, timedelta

In [0]:
os.environ["DATABRICKS_HOST"] = "{Put your Databricks Hostname here}"
os.environ["DATABRICKS_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

In [0]:
llm = ChatDatabricks(endpoint="databricks-claude-sonnet-4-5")

In [0]:
class SensorTools:

    @tool
    def get_max_sensor_value_for_a_day(sensor_id: str, date: str) -> str:
        """
        Retrieves the maximum recorded value for a specific sensor 
        from the Delta table 'sensor_readings_delta'.
        
        Args:
            sensor_id (str): Name/ID of the sensor.
            date (str): Date in 'YYYY-MM-DD' format.
        
        Returns:
            str: The maximum Temperature value as a string.
        """
        df = spark.sql(f"""
            SELECT MAX(value) AS max_value 
            FROM genai_usecases.multi_step_tools.sensor_readings_delta 
            WHERE sensor_id = '{sensor_id}' AND to_date(ts) = to_date('{date}')
        """)
        
        max_val = df.collect()[0]["max_value"]
        return str(max_val) if max_val is not None else "No data found"

    @tool
    def get_current_date() -> str:
        """
        Returns the current timestamp.

        This tool fetches the system's current local date and time. 
        It is useful for tasks where the LLM needs to reference the actual 
        current time instead of reasoning about it contextually.

        Returns:
            str: The current local timestamp in the format YYYY-MM-DD HH:MM:SS.
        """
        return datetime.now().strftime("%Y-%m-%d")


    @tool
    def get_past_date(days_before: int) -> str:
        """
        Returns the date N days before the current date.

        This tool calculates the date that occurred a specific number 
        of days before today. It is handy for temporal comparisons, 
        like fetching sensor readings or logs from a few days back.

        Args:
            days_before (int): Number of days to go back from today.
                            Default is 3.

        Returns:
            str: The date N days before today in the format YYYY-MM-DD.
        """
        past_date = datetime.now() - timedelta(days=days_before)
        return past_date.strftime("%Y-%m-%d")
    

In [0]:
tools = [SensorTools.get_max_sensor_value_for_a_day, SensorTools.get_current_date, SensorTools.get_past_date]
llm_with_tools = llm.bind_tools(tools)

In [0]:
# Step 1: Define the user query
# This time, we’re testing multi-step reasoning: the model needs to call multiple tools sequentially.
user_query = input("Enter the question: ")

# Step 2: Create an initial conversation context
# It starts with the user's question, wrapped as a HumanMessage.
messages = [SystemMessage("""You are strictly prohibited from performing any numeric or arithmetic calculations using your own reasoning or internal logic.
Do not explain results that come from your own estimation or calculation — rely solely on tool outputs for all numbers."""),HumanMessage(user_query)]

# Step 3: Run a loop until the model signals that it has finished reasoning
# The model keeps responding with tool calls until finish_reason == "stop"
stop_reason = None

while stop_reason != "stop":
    # Step 4: Invoke the LLM with the current conversation state
    ai_message = llm_with_tools.invoke(messages)

    # Capture the reason the model stopped generating text in this iteration
    stop_reason = ai_message.response_metadata.get("finish_reason", None)
    print(f"Model finish reason: {stop_reason}")

    # Add the model's message to the conversation history
    messages.append(ai_message)

    # Step 5: If the model wants to use tools (finish_reason == "tool_calls"),
    # iterate through them and execute one by one.
    if ai_message.tool_calls:
        for tool_call in ai_message.tool_calls:

            print("Current Tool Call details: ")
            print(tool_call)
            # Step 5a: Match the tool name to the correct Python function.
            # Lowercasing ensures we don’t mismatch due to naming conventions.
            selected_tool = {
                "get_max_sensor_value_for_a_day": SensorTools.get_max_sensor_value_for_a_day,
                "get_current_date": SensorTools.get_current_date,
                "get_past_date": SensorTools.get_past_date
            }[tool_call["name"].lower()]

            # Step 5b: Execute the tool and get a ToolMessage back.
            # This message structure is automatically handled by LangChain.
            tool_msg = selected_tool.invoke(tool_call)

            # Step 5c: Append the tool’s response to the conversation.
            # This allows the LLM to see its previous reasoning and tool outputs.
            messages.append(tool_msg)

            print(f"Executed Tool: {tool_call['name']}, Output: {tool_msg.content}")

    # Step 6: If the model has completed (finish_reason == "stop"),
    # it means the response is ready — no more tools needed.
    elif stop_reason == "stop":
        print("✅ Model completed reasoning — final answer below:")
        print(ai_message.content)
        break

    # Step 7: Handle unexpected finish reasons (optional but good practice)
    else:
        print(f"⚠️ Unexpected finish reason: {stop_reason}")
        break

Enter the question:  What is the max temperature for sensor_B today?

Model finish reason: tool_calls
Current Tool Call details: 
{'name': 'get_current_date', 'args': {}, 'id': 'toolu_bdrk_01CbrmTgg7CwB3cXHXgAXW5s', 'type': 'tool_call'}
Executed Tool: get_current_date, Output: 2025-10-24
Model finish reason: tool_calls
Current Tool Call details: 
{'name': 'get_max_sensor_value_for_a_day', 'args': {'sensor_id': 'sensor_B', 'date': '2025-10-24'}, 'id': 'toolu_bdrk_01BaggsHGQk3vUhcXgToyrH6', 'type': 'tool_call'}
Executed Tool: get_max_sensor_value_for_a_day, Output: 640.0
Model finish reason: stop
✅ Model completed reasoning — final answer below:
The maximum temperature for sensor_B today (2025-10-24) is **640.0**.
